In [288]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [289]:
import pandas as pd
import pandas_profiling
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re
import math
import copy
from IPython.display import display
pd.options.mode.chained_assignment = None
import random
from datetime import datetime, timedelta
import requests
from pandas_profiling import ProfileReport
#выгружаем специальный и приемлимый инструмент для разделения датасетов.
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [290]:
RANDOM_SEED = 42

In [291]:
#записываем всегда RANDOM_SEED,чтобы в дальнейшем была возможность воспроизвести код.
!pip freeze > requirements.txt

**DATA**

In [292]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [293]:
df_train.info()


In [294]:
df_train.head(5)

In [295]:
sample_submission.head(5)

In [296]:
sample_submission.info()

In [297]:
# P.S:для нормальной обработки признаков соединяем трейн и тест в один датасет.
df_train['sample'] = 1 # отмечаем трейн.
df_test['sample'] = 0 # отмечаем тест.
df_test['Rating'] = 0 # в тесте отсутствует значение Rating,на время заполним его нулём.

df = df_test.append(df_train, sort=False).reset_index(drop=True)

In [298]:
df.info()

In [299]:
df.sample(5)

In [300]:
df.Reviews[1]

**Переходим к Очистке Датасета.**

In [301]:
df.head(2)

In [302]:
#заменим наименования столбцов.
df.columns = df.columns.str.lower()
df.columns = ['restaurant_id', 'city', 'cuisine', 'ranking',
              'price', 'reviews_num', 'reviews', 'url_tag', 'id_tag', 'sample', 'rating']

**Обработка NaN**

In [303]:
df['reviews_num_isNAN'] = pd.isna(df['reviews_num']).astype('uint8')

In [304]:
sns.boxplot(df['reviews_num'])

In [305]:
df.reviews_num = df.reviews_num.fillna(df.reviews_num.mode()[0])

In [306]:
df['cuisine_isna'] = pd.isna(df['cuisine']).astype('uint8')

In [307]:
df.city.unique()

In [308]:
df.city = df.city.str.replace('Oporto', 'Porto')

In [309]:
pop = {'Paris':2125246, 'Stockholm':792593, 'London':7172091, 'Berlin':3399739, 'Munich':1259677, 'Porto':242883,
       'Milan':1256211, 'Bratislava':426091, 'Vienna':1664146, 'Rome':2546804, 'Barcelona':1605602, 'Madrid':3213271,
       'Dublin':505739, 'Brussels':1006749, 'Zurich':100000, 'Warsaw':1760800, 'Budapest':1727300, 'Copenhagen':1145804,
       'Amsterdam':743104, 'Lyon':445452, 'Hamburg':1743627, 'Lisbon':508209, 'Prague':1285995, 'Oslo':505000,
       'Helsinki':565186, 'Edinburgh':448624, 'Geneva':100001, 'Ljubljana':266845, 'Athens':720979,
       'Luxembourg':100002, 'Krakow':789590}

df['population'] = df.city.apply(lambda x: pop.get(x))
df.population = df.population.astype('int64')

In [310]:
pop = {'Paris':2125246, 'Stockholm':792593, 'London':7172091, 'Berlin':3399739, 'Munich':1259677, 'Porto':242883,
       'Milan':1256211, 'Bratislava':426091, 'Vienna':1664146, 'Rome':2546804, 'Barcelona':1605602, 'Madrid':3213271,
       'Dublin':505739, 'Brussels':1006749, 'Zurich':100000, 'Warsaw':1760800, 'Budapest':1727300, 'Copenhagen':1145804,
       'Amsterdam':743104, 'Lyon':445452, 'Hamburg':1743627, 'Lisbon':508209, 'Prague':1285995, 'Oslo':505000,
       'Helsinki':565186, 'Edinburgh':448624, 'Geneva':100001, 'Ljubljana':266845, 'Athens':720979,
       'Luxembourg':100002, 'Krakow':789590}

df['population'] = df.city.apply(lambda x: pop.get(x))
df.population = df.population.astype('int64')

In [311]:
bc = Counter(df.city).most_common(8)
biggest_cities = []
for city in bc:
    city = city[0]
    biggest_cities.append(city)

df.city = df.city.apply(lambda x: x if x in biggest_cities else 'Other')
df = pd.get_dummies(df, dummy_na=False, columns=['city'])

In [312]:
df.head (5)

In [313]:
df.sample (5)

In [314]:
df['price'].value_counts()

In [315]:
df.price = df.price.replace({'$':1, '$$ - $$$': 2, '$$$$': 3})
df.price = df.price.fillna(df.price.mean()) 
df.price = df.price.astype('int64')

In [316]:
df.cuisine.head(5)

In [317]:
def cuisine_clean(str_val):
    if pd.isnull(str_val):
        return str_val
    str_val = str_val.strip('[]')
    str_val = str_val.replace("\'", '')
    str_val = str_val.split(', ')
    return str_val

df['cuisine'] = df.cuisine.apply(cuisine_clean)

In [318]:
df.cuisine.head(5)

In [319]:
rc = df.cuisine.dropna()
csl = []
for row in rc:
    for i in row:
        csl.append(i)
sc = Counter(csl).most_common(40)

common_cuisine = []
common_cuisine.append('other')
for item in sc:
    item = item[0]
    common_cuisine.append(item)

In [320]:
df.cuisine = df.cuisine.fillna(random.choice(common_cuisine))

In [321]:
def set_common_styles(style):
    row = set()
    for i in style:
        if i in common_cuisine:
            row.add(i)
        else:
            row.add('other')
    return row

In [322]:
df.cuisine = df.cuisine.apply(set_common_styles)

In [323]:
df.cuisine.head(10)

In [324]:
def find_item(cell):
    if item in cell:
        return 1
    return 0

for item in common_cuisine:
    df[item] = df.cuisine.apply(find_item)
df.columns = df.columns.str.lower()
df.cuisine = df.cuisine.apply(len)

In [325]:
df.info()

In [326]:
df.reviews = df.reviews.fillna('[[], []]')

In [327]:
df['reviews_date'] = df.reviews.str.findall('\d{2}/\d{2}/\d{4}')

In [328]:
df.reviews_date.head(5)

In [329]:
def to_time(x):
    datetime_list = []
    if len(x) == 3:
        x = x[1:]
    if type(x) != list:
        pass
    for date in x:
        nl = datetime.strptime(date, '%m/%d/%Y')
        datetime_list.append(nl) 
    return datetime_list

In [330]:
df.reviews_date = df.reviews_date.apply(to_time)

In [331]:
df.reviews_date.head(5)

In [332]:
df['reviews_delta'] = df.reviews_date.apply(lambda x: x[0] - x[1] if len(x) == 2 else np.nan)
df.reviews_delta = df.reviews_delta.apply(lambda x: int(x.days) if not pd.isna(x) else x)
#df['reviews_delta_isNAN'] = pd.isna(df['reviews_delta']).astype('uint8')
df.reviews_delta = df.reviews_delta.fillna(df.reviews_delta.mean())

In [333]:
df.info()

In [334]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [335]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [336]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [337]:
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

In [338]:
mms = df[['ranking', 'population']]
min_max_Scaler = MinMaxScaler()
result = min_max_Scaler.fit_transform(mms)
result = pd.DataFrame(result)
result.columns = ['ranking_norm', 'population_norm']
df = pd.concat([df, result], axis=1)

In [339]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

In [340]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [341]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

In [342]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df.drop(['sample'], axis=1).corr(),)

In [343]:
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values
X = train_data.drop(['restaurant_id', 'rating', 'id_tag', 'url_tag', 'reviews', 'reviews_date','price', 'cuisine'], axis=1)

In [344]:
def StandardScaler_func(col):
    scaler = StandardScaler()
    scaler.fit(df[col])
    return scaler.transform(df[col])
for i in list(df.columns):
    df[i] = StandardScaler(i)
    if len(df[df[i].isna()]) < len(df):
        df[i] = df[i].fillna(0)

In [351]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [353]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics 

In [354]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [356]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [358]:
def classic_round(d_num):
    return int(d_num + (0.5 if d_num > 0 else -0.5))

def my_round(d_pred):
    result = classic_round(d_pred*2)/2
    if result <=5:
        return result
    else:
        return 5
    
my_vec_round = np.vectorize(my_round)

In [359]:
y_pred = my_vec_round(y_pred)

In [361]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [360]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [352]:
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [346]:
test_data.sample(10)

In [347]:
test_data = test_data.drop(['restaurant_id', 'rating', 'id_tag', 'url_tag', 'reviews', 'reviews_date','price', 'cuisine'], axis=1)

In [348]:
test_data

In [349]:
sample_submission

In [362]:
model.fit(X, y)

In [363]:
predict_submission = model.predict(test_data)

In [364]:
predict_submission = my_vec_round(predict_submission)

In [365]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)